In [89]:
import pandas as pd
import json
import synapseclient

syn = synapseclient.Synapse()
syn.login(silent=True)

gene_info = 'syn25953363'
networks = 'syn11685347'

gene_info = syn.get(gene_info)
networks = syn.get(networks)

gene_info = pd.read_feather(gene_info.path)
networks = pd.read_csv(networks.path)

In [90]:
gene_info.rename(columns={"symbol": "hgnc_symbol"}, inplace=True)
gene_info = gene_info[['ensembl_gene_id', 'hgnc_symbol']]
gene_info.head(3)

,ensembl_gene_id,hgnc_symbol
0,ENSG00000223972,DDX11L1
1,ENSG00000227232,WASH7P
2,ENSG00000278267,MIR6859-1


In [91]:
print(networks.shape)
networks.head(3)

(347419, 5)


,geneA_ensembl_gene_id,geneB_ensembl_gene_id,geneA_external_gene_name,geneB_external_gene_name,brainRegion
0,ENSG00000268903,ENSG00000150337,AL627309.7,FCGR1A,DLPFC
1,ENSG00000225972,ENSG00000229344,MTND1P23,MTCO2P12,DLPFC
2,ENSG00000225972,ENSG00000269028,MTND1P23,MTRNR2L12,DLPFC


In [92]:
networks = networks[networks['geneA_ensembl_gene_id'].isin(gene_info['ensembl_gene_id']) & networks['geneB_ensembl_gene_id'].isin(gene_info['ensembl_gene_id'])]

# unclear about the line below. Line 286 on lib.R suggests this
# networks = networks[['geneA_ensembl_gene_id', 'geneB_ensembl_gene_id', 'brainRegion']]
networks.shape

(344587, 5)

In [93]:
gene_info.drop_duplicates(inplace=True)
merged = pd.merge(left=networks, right=gene_info, left_on='geneA_ensembl_gene_id', right_on='ensembl_gene_id', how='left')

print(merged.shape)
merged.head(3)

(344600, 7)


,geneA_ensembl_gene_id,geneB_ensembl_gene_id,geneA_external_gene_name,geneB_external_gene_name,brainRegion,ensembl_gene_id,hgnc_symbol
0,ENSG00000268903,ENSG00000150337,AL627309.7,FCGR1A,DLPFC,ENSG00000268903,None
1,ENSG00000225972,ENSG00000229344,MTND1P23,MTCO2P12,DLPFC,ENSG00000225972,MTND1P23
2,ENSG00000225972,ENSG00000269028,MTND1P23,MTRNR2L12,DLPFC,ENSG00000225972,MTND1P23


In [94]:
merged = pd.merge(left=networks, right=gene_info, left_on='geneB_ensembl_gene_id', right_on='ensembl_gene_id', how='left')
# merged.rename(columns={'geneB_external_gene_name':'hgnc_symbol'})
print(merged.shape)
merged.head(3)

(344659, 7)


,geneA_ensembl_gene_id,geneB_ensembl_gene_id,geneA_external_gene_name,geneB_external_gene_name,brainRegion,ensembl_gene_id,hgnc_symbol
0,ENSG00000268903,ENSG00000150337,AL627309.7,FCGR1A,DLPFC,ENSG00000150337,FCGR1A
1,ENSG00000225972,ENSG00000229344,MTND1P23,MTCO2P12,DLPFC,ENSG00000229344,MTCO2P12
2,ENSG00000225972,ENSG00000269028,MTND1P23,MTRNR2L12,DLPFC,ENSG00000269028,MTRNR2L2


In [103]:
merged = merged[['geneA_ensembl_gene_id', 'geneB_ensembl_gene_id',
       'geneA_external_gene_name', 'geneB_external_gene_name', 'brainRegion']]
merged.to_json('../output/network.json', orient='records')